## Test workgrah

## Visualizing the WorkGraph

In [5]:
from aiida_qe_app.workflows.qeapp_workgraph import qeapp_workgraph
from aiida import load_profile, orm
from ase.build import bulk

load_profile()

si = bulk("Si")
structure = orm.StructureData(ase=si)
pw_code = orm.load_code("qe-7.2-pw@localhost")
projwfc_bands_code = orm.load_code("qe-7.2-projwfc@localhost")
projwfc_code = orm.load_code("qe-7.2-projwfc@localhost")
dos_code = orm.load_code("qe-7.2-dos@localhost")

scf_paras = {
    'SYSTEM': {
        'ecutwfc': 30,
        'ecutrho': 240,
        'smearing': 'gaussian',
        'degauss': 0.1,
    },
}
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([3, 3, 3])
metadata = {
    'options': {
        'resources': {
            'num_machines': 1,
            'num_mpiprocs_per_machine': 4,
        },
    }
}
#
parameters = {
    "workchain": {"protocol": "fast",
                  "relax_type": "positions",
                  "electronic_type": "metal",
                  "spin_type": "none",
                #   "properties": ["relax", "bands", "pdos"],
                  "properties": ["relax"]
    },
    "advanced": {
        "pw": {
            "parameters": scf_paras,
            "pseudos": {},
        },
        "initial_magnetic_moments": None,
    },
    "bands": {"projwfc_bands": False},
    "pdos": {"use_pdos_degauss": False,
             "nscf_kpoints_distance": 0.5},
    "codes": {
        "pw": {"code": pw_code.uuid, "nodes": 1, "ntasks_per_node": 1, "cpus_per_task": 1, "max_wallclock_seconds": 1800},
        "projwfc_bands": {"code": projwfc_bands_code.uuid, "nodes": 1, "ntasks_per_node": 1, "cpus_per_task": 1, "max_wallclock_seconds": 1800},
        "projwfc": {"code": projwfc_code.uuid, "nodes": 1, "ntasks_per_node": 1, "cpus_per_task": 1, "max_wallclock_seconds": 1800},
        "dos": {"code": dos_code.uuid, "nodes": 1, "ntasks_per_node": 1, "cpus_per_task": 1, "max_wallclock_seconds": 1800},
    }
}

wg = qeapp_workgraph(structure=structure,
                     parameters=parameters)

In [6]:
wg.run()

11/01/2024 11:00:34 AM <115294> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13110|WorkGraphEngine|continue_workgraph]: Continue workgraph.
11/01/2024 11:00:34 AM <115294> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13110|WorkGraphEngine|continue_workgraph]: tasks ready to run: relax
11/01/2024 11:00:34 AM <115294> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13110|WorkGraphEngine|run_tasks]: Run task: relax, type: WORKCHAIN


------------------------------------------------------------


11/01/2024 11:00:34 AM <115294> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13110|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 13112
11/01/2024 11:00:35 AM <115294> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13112|PwRelaxWorkChain|setup]: No change in volume possible for the provided base input parameters. Meta convergence is turned off.
11/01/2024 11:00:35 AM <115294> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13112|PwRelaxWorkChain|run_relax]: launching PwBaseWorkChain<13115>
11/01/2024 11:00:36 AM <115294> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13115|PwBaseWorkChain|run_process]: launching PwCalculation<13120> iteration #1
11/01/2024 11:00:42 AM <115294> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [13115|PwBaseWorkChain|results]: work chain completed after 1 iterations
11/01/2024 11:00:42 AM <115294> aiida.orm.nodes.process.workflow.

------------------------------------------------------------


{'execution_count': <Int: uuid: e7602f95-75ca-4964-bf47-d5f1532a8d7b (pk: 13127) value: 1>}

In [ ]:
from copy import deepcopy
from aiida import load_profile
from aiida.orm import Dict, KpointsData, StructureData, load_code
from ase.build import bulk
from workgraph_collections.qe.bands import bands_workgraph

load_profile()

atoms = bulk('Si')
structure_si = StructureData(ase=atoms)

pw_code = load_code('qe-7.2-pw@localhost')
scf_paras = {
    'CONTROL': {
        'calculation': 'scf',
    },
    'SYSTEM': {
        'ecutwfc': 30,
        'ecutrho': 240,
        'occupations': 'smearing',
        'smearing': 'gaussian',
        'degauss': 0.1,
    },
}
relax_paras = deepcopy(scf_paras)
relax_paras['CONTROL']['calculation'] = 'vc-relax'
bands_paras = deepcopy(scf_paras)
bands_paras['CONTROL']['calculation'] = 'bands'

kpoints = KpointsData()
kpoints.set_kpoints_mesh([3, 3, 3])
#
metadata = {
    'options': {
        'resources': {
            'num_machines': 1,
            'num_mpiprocs_per_machine': 4,
        },
    }
}

bands_inputs = {
    'relax': {
        'base': {
            'pw': {
                'parameters': Dict(relax_paras),
                'metadata': metadata,
            },
            'kpoints': kpoints,
        },
    },
    'scf': {
        'pw': {
            'parameters': Dict(scf_paras),
            'metadata': metadata,
        },
        'kpoints': kpoints,
    },
    'bands': {
        'pw': {
            'parameters': Dict(bands_paras),
            'metadata': metadata,
        },
        'kpoints': kpoints,
    },
}

wg = bands_workgraph(
    structure=structure_si,
    code=pw_code,
    inputs=bands_inputs,
    pseudo_family='SSSP/1.3/PBEsol/efficiency',
    run_relax=True,
    bands_kpoints_distance=0.1,
)
#------------------------- Submit the calculation -------------------
wg.submit(wait=True, timeout=300)
